In [2]:
%%configure -f
{"driverMemory": "4G", "driverCores": 2, "executorMemory": "12G", "executorCores": 6, "numExecutors": 3}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,,,None,✔


In [3]:
from typing import List

from pyspark import SparkFiles
from subprocess import call
import sys


def install_deps(deps: List[str]) -> None:
    call([sys.executable, '-m', 'pip', 'install', '-q', '-t', SparkFiles.getRootDirectory(), *deps])


install_deps(['numpy', 'matplotlib', 'pandas', 'scipy', 'seaborn'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
from pyspark.sql import SparkSession
spark: SparkSession

distance_calculation_df = (
    spark
    .read
    .format("mongodb")
    .option("partitioner", "com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner")
    .option("partitioner.options.partition.field", "_id")
    .option("partitioner.options.partition.size", "64")
    .option("partitioner.options.samples.per.partition", "2")
    .option("database", "enhancer3d")
    .option("collection", "distance_calculation")
    .load()
)

distance_calculation_df.rdd.getNumPartitions()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

997

In [20]:
from pyspark.sql import functions as F, types as T
import numpy as np

@F.udf(T.ArrayType(T.DoubleType()))
def diff(A, B):
    return np.abs(np.array(A) - np.array(B)).tolist()

@F.udf(T.DoubleType())
def var(A):
    return float(np.var(A))

@F.udf(T.DoubleType())
def avg(A):
    return float(np.mean(A))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
gm12878_neanderthal_df_ref = (
    distance_calculation_df
    .where(
        (F.col('_id.project_id') == '8k_models_project_GM12878')
        & (F.col('_id.ensemble_id').startswith('models3D_GM12878_Nean_models3D_GM12878_Nean_results'))
        & (F.col('gene_type') == 'protein_coding')
    )
)

gm12878_denisovan_df_ref = (
    distance_calculation_df
    .where(
        (F.col('_id.project_id') == '8k_models_project_GM12878')
        & (F.col('_id.ensemble_id').startswith('models3D_GM12878_Deni_models3D_GM12878_Deni_results'))
        & (F.col('gene_type') == 'protein_coding')
    )
)

hffc6_neanderthal_df_ref = (
    distance_calculation_df
    .where(
        (F.col('_id.project_id') == '8k_models_project_HFFC6')
        & (F.col('_id.ensemble_id').startswith('models3D_HFFC6_Nean_models3D_HFFC6_Nean_results'))
        & (F.col('gene_type') == 'protein_coding')
    )
)

hffc6_denisovan_df_ref = (
    distance_calculation_df
    .where(
        (F.col('_id.project_id') == '8k_models_project_HFFC6')
        & (F.col('_id.ensemble_id').startswith('models3D_HFFC6_Deni_models3D_HFFC6_Deni_results'))
        & (F.col('gene_type') == 'protein_coding')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%pretty
gm12878_neanderthal_df_ref.show(n=1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
gm12878_neanderthal_genes_with_average_distance_df = (
    gm12878_neanderthal_df_ref
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        'avg_dist'
    )
    .groupBy('gene_id')
    .agg(
        F.avg('avg_dist').alias('avg_dist_per_gene')
    )
    .orderBy(F.desc('avg_dist_per_gene'))
    .alias("gm12878")
)

hffc6_neanderthal_genes_with_average_distance_df = (
    hffc6_neanderthal_df_ref
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        'avg_dist'
    )
    .groupBy('gene_id')
    .agg(
        F.avg('avg_dist').alias('avg_dist_per_gene')
    )
    .orderBy(F.desc('avg_dist_per_gene'))
    .alias("hffc6")
)

gm12878_hffc6_neanderthal_genes_with_biggest_distance_difference_df = (
    gm12878_neanderthal_genes_with_average_distance_df
    .join(
        hffc6_neanderthal_genes_with_average_distance_df,
        on=['gene_id'],
        how='inner'
    )
    .select(
        'gene_id',
        F.col('gm12878.avg_dist_per_gene').alias('dist_diff_avg_gm12878'),
        F.col('hffc6.avg_dist_per_gene').alias('dist_diff_avg_hffc6'),
        F.abs((F.col('gm12878.avg_dist_per_gene') - F.col('hffc6.avg_dist_per_gene'))).alias('dist_diff_avg'),
    )
    .orderBy(F.desc('dist_diff_avg'))
    # .limit(100)
)

gm12878_denisovan_genes_with_average_distance_df = (
    gm12878_denisovan_df_ref
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        'avg_dist'
    )
    .groupBy('gene_id')
    .agg(
        F.avg('avg_dist').alias('avg_dist_per_gene')
    )
    .orderBy(F.desc('avg_dist_per_gene'))
    .alias("gm12878")
)

hffc6_denisovan_genes_with_average_distance_df = (
    hffc6_denisovan_df_ref
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        'avg_dist'
    )
    .groupBy('gene_id')
    .agg(
        F.avg('avg_dist').alias('avg_dist_per_gene')
    )
    .orderBy(F.desc('avg_dist_per_gene'))
    .alias("hffc6")
)

gm12878_hffc6_denisovan_genes_with_biggest_distance_difference_df = (
    gm12878_denisovan_genes_with_average_distance_df
    .join(
        hffc6_denisovan_genes_with_average_distance_df,
        on=['gene_id'],
        how='inner'
    )
    .select(
        'gene_id',
        F.col('gm12878.avg_dist_per_gene').alias('dist_diff_avg_gm12878'),
        F.col('hffc6.avg_dist_per_gene').alias('dist_diff_avg_hffc6'),
        F.abs((F.col('gm12878.avg_dist_per_gene') - F.col('hffc6.avg_dist_per_gene'))).alias('dist_diff_avg'),
    )
    .orderBy(F.desc('dist_diff_avg'))
    # .limit(100)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
%%pretty
gm12878_hffc6_neanderthal_genes_with_biggest_distance_difference_df.show(n=5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gene_id,dist_diff_avg_gm12878,dist_diff_avg_hffc6,dist_diff_avg
ENSG00000130558.20,6.204862315576155,39.31081470366447,33.10595238808831
ENSG00000169717.7,6.218723866427056,39.23214709069118,33.01342322426412
ENSG00000198967.5,11.19432799670161,42.023285781280904,30.828957784579295
ENSG00000127903.14,38.720140565655186,8.26742925621845,30.452711309436737
ENSG00000117174.11,39.42625308036804,9.091781710678676,30.334471369689368


In [11]:
# to pandas and plot
import pandas as pd

gm12878_hffc6_neanderthal_genes_with_biggest_distance_difference_df_pd = (
    gm12878_hffc6_neanderthal_genes_with_biggest_distance_difference_df
    .toPandas()
)

# save the dataframe to a csv file
gm12878_hffc6_neanderthal_genes_with_biggest_distance_difference_df_pd.to_csv(
    '/work/playground/gm12878_hffc6_neanderthal_genes_with_biggest_distance_difference_df.csv',
    index=False
)

gm12878_hffc6_denisovan_genes_with_biggest_distance_difference_df_pd = (
    gm12878_hffc6_denisovan_genes_with_biggest_distance_difference_df
    .toPandas()
)

# save the dataframe to a csv file
gm12878_hffc6_denisovan_genes_with_biggest_distance_difference_df_pd.to_csv(
    '/work/playground/gm12878_hffc6_denisovan_genes_with_biggest_distance_difference_df.csv',
    index=False
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…